In [1]:
pip install yfinance pandas numpy matplotlib seaborn scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Clean Stock Recommendation System - Simple Output Format
# User inputs stock symbol and gets clean buy/sell recommendations

import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.feature_selection import SelectKBest, f_regression
import warnings
warnings.filterwarnings('ignore')

In [3]:
class StockRecommendationSystem:
    def __init__(self):
        """Initialize the Stock Recommendation System"""
        self.symbol = None
        self.data = None
        self.models = {}
        self.scalers = {}
        
    def fetch_data(self, symbol, period="2y"):
        """Fetch and validate stock data"""
        self.symbol = symbol.upper()
        
        try:
            stock = yf.Ticker(self.symbol)
            self.data = stock.history(period=period)
            
            if self.data.empty:
                raise ValueError(f"No data found for {self.symbol}")
                
            return True
        except:
            return False
    
    def create_technical_indicators(self):
        """Create technical indicators"""
        df = self.data.copy()
        
        # Moving Averages
        df['MA_5'] = df['Close'].rolling(window=5).mean()
        df['MA_10'] = df['Close'].rolling(window=10).mean()
        df['MA_20'] = df['Close'].rolling(window=20).mean()
        df['MA_50'] = df['Close'].rolling(window=50).mean()
        
        # Exponential Moving Averages
        df['EMA_12'] = df['Close'].ewm(span=12).mean()
        df['EMA_26'] = df['Close'].ewm(span=26).mean()
        
        # MACD
        df['MACD'] = df['EMA_12'] - df['EMA_26']
        df['MACD_signal'] = df['MACD'].ewm(span=9).mean()
        
        # RSI
        delta = df['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        df['RSI'] = 100 - (100 / (1 + rs))
        
        # Bollinger Bands
        df['BB_upper'] = df['MA_20'] + (df['Close'].rolling(window=20).std() * 2)
        df['BB_lower'] = df['MA_20'] - (df['Close'].rolling(window=20).std() * 2)
        df['BB_position'] = (df['Close'] - df['BB_lower']) / (df['BB_upper'] - df['BB_lower'])
        
        # Price features
        df['Price_Change'] = df['Close'].pct_change()
        df['High_Low_Ratio'] = df['High'] / df['Low']
        df['Volume_MA'] = df['Volume'].rolling(window=10).mean()
        df['Volume_Ratio'] = df['Volume'] / df['Volume_MA']
        df['Volatility'] = df['Price_Change'].rolling(window=10).std()
        
        # Lagged features
        for lag in [1, 2, 3, 5]:
            df[f'Close_lag_{lag}'] = df['Close'].shift(lag)
            df[f'Volume_lag_{lag}'] = df['Volume'].shift(lag)
        
        self.data = df
    
    def prepare_and_train_models(self):
        """Prepare features and train models"""
        df = self.data.copy()
        df['Target'] = df['Close'].shift(-1)  # Next day price
        
        # Select features
        feature_cols = [col for col in df.columns if col not in ['Target', 'Dividends', 'Stock Splits']]
        df_clean = df.dropna()
        
        X = df_clean[feature_cols]
        y = df_clean['Target']
        
        # Feature selection
        selector = SelectKBest(score_func=f_regression, k=15)
        X_selected = selector.fit_transform(X, y)
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X_selected, y, test_size=0.2, random_state=42, shuffle=False
        )
        
        # Scale features
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        self.scalers['standard'] = scaler
        
        # Train models
        models = {
            'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
            'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
            'Linear Regression': LinearRegression(),
            'SVR': SVR(kernel='rbf', C=100, gamma=0.1)
        }
        
        results = {}
        for name, model in models.items():
            if name in ['Linear Regression', 'SVR']:
                model.fit(X_train_scaled, y_train)
                y_pred = model.predict(X_test_scaled)
            else:
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)
            
            r2 = r2_score(y_test, y_pred)
            results[name] = {'model': model, 'r2': r2}
        
        self.models = results
        self.X_features = X
        self.selector = selector
        return results
    
    def get_recommendation(self):
        """Generate clean recommendation output"""
        # Get best model
        best_model_name = max(self.models.keys(), key=lambda x: self.models[x]['r2'])
        best_model = self.models[best_model_name]['model']
        best_r2 = self.models[best_model_name]['r2']
        
        # Predict next price
        last_features = self.X_features.iloc[-1:].values
        last_features_selected = self.selector.transform(last_features)
        
        if best_model_name in ['Linear Regression', 'SVR']:
            last_features_scaled = self.scalers['standard'].transform(last_features_selected)
            prediction = best_model.predict(last_features_scaled)[0]
        else:
            prediction = best_model.predict(last_features_selected)[0]
        
        current_price = self.data['Close'].iloc[-1]
        expected_change = ((prediction - current_price) / current_price) * 100
        
        # Get technical signals
        latest = self.data.iloc[-1]
        signals_score = 0
        
        # ML Prediction signal
        if expected_change > 2:
            signals_score += 2
        elif expected_change > 0.5:
            signals_score += 1
        elif expected_change < -2:
            signals_score -= 2
        elif expected_change < -0.5:
            signals_score -= 1
        
        # Moving Average signal
        if current_price > latest['MA_20'] > latest['MA_50']:
            signals_score += 1
        elif current_price < latest['MA_20'] < latest['MA_50']:
            signals_score -= 1
        
        # RSI signal
        rsi = latest['RSI']
        if rsi < 30:
            signals_score += 1
        elif rsi > 70:
            signals_score -= 1
        
        # MACD signal
        if latest['MACD'] > latest['MACD_signal']:
            signals_score += 1
        else:
            signals_score -= 1
        
        # Bollinger Bands signal
        bb_pos = latest['BB_position']
        if bb_pos < 0.2:
            signals_score += 1
        elif bb_pos > 0.8:
            signals_score -= 1
        
        # Volume signal
        if latest['Volume_Ratio'] > 1.2:
            signals_score += 1
        elif latest['Volume_Ratio'] < 0.8:
            signals_score -= 1
        
        # Generate recommendation
        if signals_score >= 3:
            recommendation = "🟢 STRONG BUY"
            action = "Excellent buying opportunity - strong bullish signals"
        elif signals_score >= 1:
            recommendation = "🟢 BUY"
            action = "Good buying opportunity - more bullish than bearish"
        elif signals_score >= -1:
            recommendation = "🟡 HOLD"
            action = "Mixed signals - wait for clearer direction"
        elif signals_score >= -3:
            recommendation = "🔴 SELL"
            action = "Consider selling - bearish signals dominating"
        else:
            recommendation = "🔴 STRONG SELL"
            action = "Strong sell signal - exit positions recommended"
        
        return {
            'symbol': self.symbol,
            'current_price': current_price,
            'predicted_price': prediction,
            'expected_change': expected_change,
            'model_accuracy': best_r2,
            'recommendation': recommendation,
            'action': action
        }

def get_stock_recommendation(symbol):
    """Main function to get clean stock recommendation"""
    system = StockRecommendationSystem()
    
    # Fetch data
    if not system.fetch_data(symbol):
        return f"❌ Error: Could not find data for {symbol.upper()}. Please check the symbol."
    
    try:
        # Process data and train models
        system.create_technical_indicators()
        system.prepare_and_train_models()
        
        # Get recommendation
        result = system.get_recommendation()
        
        # Format clean output
        output = f"""🎯 STOCK RECOMMENDATION FOR {result['symbol']}
Current Price: ${result['current_price']:.2f}
Predicted Price: ${result['predicted_price']:.2f}
Expected Change: {result['expected_change']:+.1f}%
Model Accuracy: {result['model_accuracy']:.0%}

{result['recommendation']}
💡 ACTION: {result['action']}"""
        
        return output
        
    except Exception as e:
        return f"❌ Error analyzing {symbol.upper()}: {str(e)}"

In [4]:
def interactive_mode():
    """Interactive mode for continuous stock analysis"""
    print("🚀 STOCK RECOMMENDATION SYSTEM")
    print("="*50)
    print("Enter stock symbols for AI-powered recommendations!")
    print("Examples: AAPL, GOOGL, MSFT, TSLA, AMZN")
    print("Type 'quit' to exit\n")
    
    while True:
        try:
            symbol = input("💼 Enter stock symbol: ").strip()
            
            if symbol.lower() in ['quit', 'exit', 'q']:
                print("👋 Thank you for using the Stock Recommendation System!")
                break
            
            if not symbol:
                print("⚠️  Please enter a valid stock symbol\n")
                continue
            
            print(f"\n🔍 Analyzing {symbol.upper()}...\n")
            result = get_stock_recommendation(symbol)
            print(result)
            print("\n" + "="*50 + "\n")
                
        except KeyboardInterrupt:
            print("\n👋 Thank you for using the Stock Recommendation System!")
            break
        except Exception as e:
            print(f"❌ Error: {str(e)}\n")

def quick_analysis(symbol):
    """Quick single stock analysis"""
    print(get_stock_recommendation(symbol))

def analyze_multiple_stocks(symbols):
    """Analyze multiple stocks and show summary"""
    print("📊 PORTFOLIO ANALYSIS")
    print("="*50)
    
    results = []
    for symbol in symbols:
        print(f"\n🔍 Analyzing {symbol}...")
        result = get_stock_recommendation(symbol)
        print(result)
        print("-" * 50)
        results.append(result)
    
    return results

In [8]:
# Example usage
if __name__ == "__main__":
    # Interactive mode
    interactive_mode()
    
    # For single stock analysis, uncomment below:
    # quick_analysis("AAPL")
    
    # For multiple stocks analysis, uncomment below:
    # stocks = ["AAPL", "GOOGL", "MSFT", "TSLA", "NVDA"]
    # analyze_multiple_stocks(stocks)

🚀 STOCK RECOMMENDATION SYSTEM
Enter stock symbols for AI-powered recommendations!
Examples: AAPL, GOOGL, MSFT, TSLA, AMZN
Type 'quit' to exit


🔍 Analyzing AAPL...

🎯 STOCK RECOMMENDATION FOR AAPL
Current Price: $203.27
Predicted Price: $200.93
Expected Change: -1.1%
Model Accuracy: 89%

🔴 SELL
💡 ACTION: Consider selling - bearish signals dominating


⚠️  Please enter a valid stock symbol

👋 Thank you for using the Stock Recommendation System!
